In [17]:
# Los datasets noisy son aquellos en donde de forma aleatoria se modifican el 10% de las solicitudes.

import pandas as pd
import numpy as np

def introducir_ruido(df, columna='ACTION', porcentaje=0.1, random_state=None):
    """
    Cambia un porcentaje de valores en la columna 'ACTION' de 0 a 1 y de 1 a 0.
    Mantiene la proporción original de aparición de 0s y 1s.

    Parámetros:
    - df: DataFrame original
    - columna: nombre de la columna binaria
    - porcentaje: proporción de valores a modificar (entre 0 y 1)
    - random_state: semilla para reproducibilidad

    Retorna:
    - df_modificado: copia del DataFrame con ruido introducido
    """
    np.random.seed(random_state)
    df_modificado = df.copy()

    # Índices de valores 0 y 1
    indices_0 = df_modificado[df_modificado[columna] == 0].index
    indices_1 = df_modificado[df_modificado[columna] == 1].index

    # Cantidad a modificar
    n_0 = int(len(indices_0) * porcentaje)
    n_1 = int(len(indices_1) * porcentaje)

    # Selección aleatoria de índices para cambiar
    indices_0_a_cambiar = np.random.choice(indices_0, size=n_0, replace=False)
    indices_1_a_cambiar = np.random.choice(indices_1, size=n_1, replace=False)

    # Aplicar cambios
    df_modificado.loc[indices_0_a_cambiar, columna] = 1
    df_modificado.loc[indices_1_a_cambiar, columna] = 0

    return df_modificado

def crear_version_sparse(df, columna='ACTION', porcentaje=0.1, random_state=None):
    """
    Crea una versión 'sparse' del DataFrame seleccionando un porcentaje de registros
    manteniendo la proporción original de la columna binaria 'ACTION'.

    Parámetros:
    - df: DataFrame original
    - columna: nombre de la columna binaria
    - porcentaje: proporción de registros a conservar (entre 0 y 1)
    - random_state: semilla para reproducibilidad

    Retorna:
    - df_sparse: DataFrame reducido con proporción original de la columna
    """
    np.random.seed(random_state)
    
    # Separar por clase
    df_0 = df[df[columna] == 0]
    df_1 = df[df[columna] == 1]

    # Calcular cuántos registros conservar de cada clase
    n_0 = int(len(df_0) * porcentaje)
    n_1 = int(len(df_1) * porcentaje)

    # Seleccionar aleatoriamente
    df_0_sparse = df_0.sample(n=n_0, random_state=random_state)
    df_1_sparse = df_1.sample(n=n_1, random_state=random_state)

    # Combinar y reordenar
    df_sparse = pd.concat([df_0_sparse, df_1_sparse]).sample(frac=1, random_state=random_state).reset_index(drop=True)

    return df_sparse


In [4]:
FILE_NAME = "/Users/ddiaz/Documents/code/phd-thesis-lab/12-third_year/00-Data/01-AMZ/01-DistributionsCSV/AMZ-MOD.csv"
df = pd.read_csv(FILE_NAME)
df = df.drop_duplicates()

In [16]:
introducir_ruido(df, "ACTION", 0.1)

,ACTION,RID,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,uname
0,0,12611,85475,117961,118300,123472,117905,117906,290919,117908,0
1,1,10205,1540,117961,118343,123125,118536,118536,308574,118539,1
2,1,12267,14457,118219,118220,117884,117879,267952,19721,117880,2
3,1,12176,5396,117961,118343,119993,118321,240983,290919,118322,3
4,1,13177,5905,117929,117930,119569,119323,123932,19793,119325,4
...,...,...,...,...,...,...,...,...,...,...,...
32764,1,10621,16971,117961,118300,119993,118321,240983,290919,118322,4024
32765,1,10723,311198,91261,118026,122392,121143,173805,249618,121145,4797
32766,1,11951,28805,117961,118327,120299,124922,152038,118612,124924,6807
32767,1,15643,55643,118256,118257,117945,280788,280788,292795,119082,1264


In [21]:
crear_version_sparse(df, porcentaje=.90)

,ACTION,RID,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE,uname
0,1,15659,21823,117961,118446,118684,117905,117906,290919,117908,929
1,1,12398,2144,118582,120216,117878,119192,135350,119184,119194,4580
2,1,11825,5524,117961,118343,118514,117905,240983,290919,117908,691
3,1,9826,50318,117959,117960,117912,117885,117913,117887,117888,3344
4,1,10373,1553,117961,118225,123173,119093,120324,119095,119096,3800
...,...,...,...,...,...,...,...,...,...,...,...
29486,1,14864,84772,117961,118327,121694,122022,306795,119221,122024,794
29487,1,14755,25262,117961,118327,118320,307024,159162,118331,118332,3124
29488,1,9755,5490,117961,117962,5488,120618,304465,118643,120619,797
29489,1,13651,4931,118290,118291,120026,118422,243787,118424,118425,1079
